In [30]:
import pandas as pd
data = pd.read_csv('../CSV/FullMain.csv')
data[data['<TICKER>'] == 'ALRS'].head()

,<TICKER>,<DATETIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
7790,ALRS,2021.01.11 11:00:00,104.59,105.48,102.67,105.26,4359830
7791,ALRS,2021.01.11 12:00:00,105.26,105.59,104.56,104.85,2504390
7792,ALRS,2021.01.11 13:00:00,104.81,104.92,103.70,104.46,1887640
7793,ALRS,2021.01.11 14:00:00,104.45,105.19,104.39,104.83,3060770
7794,ALRS,2021.01.11 15:00:00,104.83,107.27,104.32,106.35,7179830


In [2]:
!pip install catboost

In [47]:
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, mean_absolute_error, r2_score

results = {}

tickers = data['<TICKER>'].unique()

for ticker in tickers:

    ticker_data = data[data['<TICKER>'] == ticker]
    ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
    ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
    ticker_data['prev_close'] = ticker_data['<CLOSE>'].shift(14)
    ticker_data['prev_low'] = ticker_data['<LOW>'].shift(14)
    ticker_data['prev_vol'] = ticker_data['<VOL>'].shift(14)
    ticker_data = ticker_data.drop(ticker_data.index[:14])
    ticker_data = ticker_data.reset_index(drop=True)

    X = ticker_data[['prev_open', 'prev_high', 'prev_close', 'prev_low', 'prev_vol', '<OPEN>']]
    y = ticker_data['<CLOSE>']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, stratify = None)


    model = CatBoostRegressor(iterations=1000, learning_rate=0.3, depth=6, verbose=0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mae_train = mean_absolute_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    results[ticker] = {'mean_absolute_error': mae, 'r2_score': r2, 'mean_absolute_error_train': mae_train, 'r2_score_train': r2_train, 'std': np.std(y_test)}


results

/tmp/ipykernel_107919/3250448349.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
/tmp/ipykernel_107919/3250448349.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
/tmp/ipykernel_107919/3250448349.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

{'AFLT': {'mean_absolute_error': 0.3777412829658047,
  'r2_score': 0.9856673371883757,
  'mean_absolute_error_train': 0.09363193791969363,
  'r2_score_train': 0.9999510163276278,
  'std': 5.087194581575565},
 'ALRS': {'mean_absolute_error': 0.28708494239507015,
  'r2_score': 0.9846635221870487,
  'mean_absolute_error_train': 0.23151678509568463,
  'r2_score_train': 0.9998610026397822,
  'std': 3.138879718111981},
 'GAZP': {'mean_absolute_error': 0.9762353004259305,
  'r2_score': 0.9637892398154639,
  'mean_absolute_error_train': 0.52752731481039,
  'r2_score_train': 0.9998583128966503,
  'std': 8.252379890586251},
 'GMKN': {'mean_absolute_error': 47.40800716681256,
  'r2_score': 0.9753723473104348,
  'mean_absolute_error_train': 39.45145653063679,
  'r2_score_train': 0.999840254503682,
  'std': 399.7789056874269},
 'HYDR': {'mean_absolute_error': 0.0020992600319160093,
  'r2_score': 0.9890686833310607,
  'mean_absolute_error_train': 0.0010470717536090067,
  'r2_score_train': 0.99902118

In [3]:
import pandas as pd
import warnings
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, recall_score, precision_score, mean_absolute_error, r2_score

warnings.filterwarnings('ignore')

data = pd.read_csv('../CSV/FullMain.csv')
results = {}

tickers = data['<TICKER>'].unique()

for ticker in tickers:

    ticker_data = data[data['<TICKER>'] == ticker]
    ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
    ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
    ticker_data['prev_close'] = ticker_data['<CLOSE>'].shift(14)
    ticker_data['prev_low'] = ticker_data['<LOW>'].shift(14)
    ticker_data['prev_vol'] = ticker_data['<VOL>'].shift(14)
    ticker_data = ticker_data.drop(ticker_data.index[:14])
    ticker_data = ticker_data.reset_index(drop=True)

    X = ticker_data[['prev_open', 'prev_high', 'prev_close', 'prev_low', 'prev_vol', '<OPEN>']]
    y = ticker_data['<CLOSE>']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, stratify = None)


    model = CatBoostRegressor(iterations=1000, learning_rate=0.3, depth=6, verbose=0)
    params = {
        'iterations': [300, 500, 1000, 2000],
        'learning_rate': [0.1, 0.2, 0.3, 0.4],
        'depth': [4, 5, 6, 7]
    }
    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_absolute_error', cv=3, verbose=1)

    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_

    print(f"Best parameters for {ticker}: {best_params}")
#     y_pred = model.predict(X_test)
#     y_pred_train = model.predict(X_train)
#
#     mae = mean_absolute_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
#
#     mae_train = mean_absolute_error(y_train, y_pred_train)
#     r2_train = r2_score(y_train, y_pred_train)
#
#     results[ticker] = {'mean_absolute_error': mae, 'r2_score': r2, 'mean_absolute_error_train': mae_train, 'r2_score_train': r2_train, 'std': np.std(y_test)}
#
#
# results

Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for AFLT: {'depth': 4, 'iterations': 300, 'learning_rate': 0.1}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for ALRS: {'depth': 4, 'iterations': 500, 'learning_rate': 0.4}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for GAZP: {'depth': 5, 'iterations': 300, 'learning_rate': 0.2}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for GMKN: {'depth': 5, 'iterations': 2000, 'learning_rate': 0.4}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for HYDR: {'depth': 5, 'iterations': 500, 'learning_rate': 0.2}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for IRAO: {'depth': 5, 'iterations': 500, 'learning_rate': 0.1}
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters for LKOH: {'depth': 4, 'iterations': 1000, 'learning_rate': 0.2}
Fitt

In [4]:
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, mean_absolute_error, r2_score

results = {}

tickers = data['<TICKER>'].unique()

for ticker in tickers:

    ticker_data = data[data['<TICKER>'] == ticker]
    ticker_data['prev_open'] = ticker_data['<OPEN>'].shift(14)
    ticker_data['prev_high'] = ticker_data['<HIGH>'].shift(14)
    ticker_data['prev_close'] = ticker_data['<CLOSE>'].shift(14)
    ticker_data['prev_low'] = ticker_data['<LOW>'].shift(14)
    ticker_data['prev_vol'] = ticker_data['<VOL>'].shift(14)
    ticker_data = ticker_data.drop(ticker_data.index[:14])
    ticker_data = ticker_data.reset_index(drop=True)

    X = ticker_data[['prev_open', 'prev_high', 'prev_close', 'prev_low', 'prev_vol', '<OPEN>']]
    y = ticker_data['<CLOSE>']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = False, stratify = None)


    model = CatBoostRegressor(iterations=300, learning_rate=0.1, depth=4, verbose=0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mae_train = mean_absolute_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    results[ticker] = {'mean_absolute_error': mae, 'r2_score': r2, 'mean_absolute_error_train': mae_train, 'r2_score_train': r2_train, 'std': np.std(y_test)}


results

{'AFLT': {'mean_absolute_error': 0.40144246370544995,
  'r2_score': 0.9845076012369663,
  'mean_absolute_error_train': 0.21870950692660798,
  'r2_score_train': 0.9995648207779766,
  'std': 5.087194581575565},
 'ALRS': {'mean_absolute_error': 0.2764953079695766,
  'r2_score': 0.9866112614316581,
  'mean_absolute_error_train': 0.5221712305823213,
  'r2_score_train': 0.9990938960437813,
  'std': 3.138879718111981},
 'GAZP': {'mean_absolute_error': 1.1522047359361045,
  'r2_score': 0.9569020787920358,
  'mean_absolute_error_train': 1.312091110454802,
  'r2_score_train': 0.9986964395014806,
  'std': 8.252379890586251},
 'GMKN': {'mean_absolute_error': 58.00291902611867,
  'r2_score': 0.9662684254706873,
  'mean_absolute_error_train': 85.91532301132686,
  'r2_score_train': 0.9990578794218509,
  'std': 399.7789056874269},
 'HYDR': {'mean_absolute_error': 0.0018422914629314148,
  'r2_score': 0.9906029552482034,
  'mean_absolute_error_train': 0.0022895584182962662,
  'r2_score_train': 0.9940309